# GDSC Certificates Generation + Mail

Steps
1. Update names string
2. Update emails string
3. Give answers for the required prompts

Made with Python <3 by Ahmed Thahir

In [1]:
names = """
Ahmed Thahir
"""

In [2]:
emails = """
f20200198@dubai.bits-pilani.ac.in
"""

In [3]:
event_name = input("Event Name ")
event_date = input("Event Date (YYYY-MM-DD) ")

from datetime import datetime
event_date = datetime.strptime(event_date, '%Y-%m-%d').strftime("%b %d, %Y")

Event Name ensetr
Event Date (YYYY-MM-DD) 2022-10-10


In [4]:
def input_event_type():
    event_type = input("""
What kind of event is this?

Enter 'W' for Workshop
or    'C' for Competition
""")
    
    if (event_type not in ['W', 'C']):
        print("Select 'W' or 'C'")
        event_type = input_event_type()
    return event_type
    
event_type = input_event_type()


What kind of event is this?

Enter 'W' for Workshop
or    'C' for Competition
W


In [5]:
names = list(filter(None, names.split("\n")))
emails = list(filter(None, emails.split("\n")))

if(len(names)!=len(emails)):
    raise ValueError('Unequal number of names and emails')

In [6]:
from getpass import getpass
sender_password = getpass("Password: ")

Password: ········


In [7]:
sender_mail = "gdsc@dubai.bits-pilani.ac.in"

In [8]:
# just check if password is right
import smtplib
with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
    server.login(sender_mail, sender_password)

# Generate Certificates

In [9]:
from PIL import Image, ImageDraw, ImageFont

img = Image.open('template.png')
W, H = img.size

In [10]:
def draw_event_details(img, event_type, event_name, event_date):
    im = img.copy()

    
    title = "CERTIFICATE OF "
    text = "has "
    if event_type == 'W':
        # Workshop
        title += "COMPLETION"
        text += "successfully completed the workshop -"
    elif event_type == 'C':
        # Competition
        title += "PARTICIPATION"
        text += "participated in"
    
    text += f" {event_name}\nheld on {event_date}"
    
    # title
    title_font = ImageFont.truetype("fonts/Product Sans Bold.ttf", W//30)
    draw = ImageDraw.Draw(im)
    w, h = draw.textsize(title, title_font)
    
    draw.text(
        (
            (W - w)//2,
            (H - h*0.8)/3.5
        ),
        title,
        font = title_font,
        align = "center",
        fill = (235, 66, 52)
    )
    
    # text
    text_font = ImageFont.truetype("fonts/Product Sans Regular.ttf", W//45)
    draw = ImageDraw.Draw(im)
    w, h = draw.textsize(text, text_font)
    
    draw.text(
        (
            (W - w)/2,
            (H - h*0.8)/1.5
        ),
        text,
        font = text_font,
        align = "center",
        fill = (90, 90, 90)
    )

    if im.mode == 'RGBA':
        im = im.convert('RGB')
    return im

img = draw_event_details(img, event_type, event_name, event_date)
# img.show()

<ipython-input-10-7f3614c7ac0e>:21: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  w, h = draw.textsize(title, title_font)
<ipython-input-10-7f3614c7ac0e>:37: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  w, h = draw.textsize(text, text_font)


In [11]:
def draw_name(img, name, y_offset=0):
    im = img.copy()
    font = ImageFont.truetype("fonts/Product Sans Bold.ttf", W//18)
    
    draw = ImageDraw.Draw(im)
    w, h = draw.textsize(name, font)   
    
    draw.text(
        (
            (W - w)//2,
            (H - h*0.8)//2 + y_offset
        ),
        name,
        font=font,
        align="center",
        fill=(50, 50, 50)
    )

    #im.show()

    if im.mode == 'RGBA':
      im = im.convert('RGB')

    im.save("output/" + name + ".pdf")

for name in names:
    draw_name(img, name)

<ipython-input-11-1e9d2592ec10>:6: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  w, h = draw.textsize(name, font)


# Send Mails

In [12]:
from email.message import EmailMessage
import imghdr

In [13]:
def gen_message(name, receiver_mail, sender_mail):
    msg = EmailMessage()
    msg['From'] = sender_mail
    msg['To'] = receiver_mail
    
    first_name = name.split(" ")[0]
    msg['Subject'] = f"{first_name} - GDSC {event_name} Certificate"

    body = f"""Hi there {first_name}!
Here is your certificate for attending {event_name} event by Google Developer Student Clubs held on {event_date}.
We hope you enjoyed it, and can't wait to see you at future events as well.

Regards
GDSC Team"""
    
    msg.set_content(body)
    
    file_name = "output/" + name + ".pdf"
    with open(file_name, 'rb') as content_file:
        content = content_file.read()
        msg.add_attachment(
            content,
            maintype='application/pdf',
            subtype='pdf',
            filename = name + f" GDSC {event_name} Certificate.pdf"
        )
    
    return msg

In [14]:
msgs = []
for name, email in zip(names, emails):
    msgs.append(gen_message(name, email, sender_mail))

In [15]:
with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
    server.login(sender_mail, sender_password)

    for msg in msgs:
        server.send_message(msg)